## OONI data analysis case study for IMAP - Myanmar

### Downloading the data

We offer a tool called oonidata (that's currently in BETA), which can be installed by running:
```
pip install oonidata
```

To download all OONI data for this example notebook, run the following command:
```
oonidata sync --country-codes MY TL MM IN VN KH PH TH ID HK --since 2022-07-01 --until 2023-07-01 --output-dir ~/projects/imap/ooni-data/ --test-name webconnectivity
```



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from urllib.parse import urlencode, quote, urlparse

from tqdm import tqdm
tqdm.pandas()

### OONI Explorer utility functions

Below are a couple of useful utility functions when dealing with measurements. They take a dataframe row and return (or print) the OONI Explorer URL. This is useful to get a link to OONI explorer to more easily inspect the raw measurement to better understand what is going on.

In [2]:
def get_explorer_url(e):
    query = ''
    if 'input' in e.keys() and e['input']:
        query = '?input={}'.format(quote(e['input'], safe=''))
    return 'https://explorer.ooni.org/measurement/{}{}'.format(e['report_id'], query)
    
def print_explorer_url(e):
    print(get_explorer_url(e))

# Extracting metadata from raw measurements

The OONI raw data is very rich, but for most analysis use-cases you just need a subset of the fields or some value that is derived from them.

Below are functions that will extract all the metadata we care about from the web_connectivity test.



In [3]:
import requests
from base64 import b64decode
import hashlib
import json
import re

def get_raw_measurement(row):
    r = requests.get("https://api.ooni.io/api/v1/measurement_meta", params={
        'report_id':row['report_id'],
        'input': row['input'],
        'full': True
    })
    j = r.json()
    return json.loads(j['raw_measurement'])

def get_resolved_ips(msmt):
    queries = msmt['test_keys'].get('queries', [])
    if not queries:
        return ''
    answers = queries[0].get('answers', [])
    if not answers:
        return []
    
    ip_list = []
    for a in answers:
        ip = a.get('ipv4', '')
        if ip:
            ip_list.append(ip)
    return ip_list

def get_control_failure(msmt):
    if 'test_keys' not in msmt:
        return 'missing_test_keys'
    return msmt['test_keys']['control_failure']

def get_test_keys_blocking(msmt):
    return str(msmt['test_keys']['blocking'])

def get_http_experiment_failure(msmt):
    return str(msmt['test_keys']['http_experiment_failure'])

def get_resolver_info(msmt):
    return {
        'resolver_ip': msmt.get('resolver_ip', ''),
        'resolver_asn': msmt.get('resolver_asn', ''),
        'resolver_network_name': msmt.get('resolver_network_name', '')
    }

def get_network_events(msmt):
    return msmt['test_keys'].get('network_events', [])

def get_tcp_connect(msmt):
    return msmt['test_keys'].get('tcp_connect', [])

def decode_body(body):
    if body is None:
        return ''
    if isinstance(body, dict):
        raw_body = b64decode(body['data'])
        try:
            return raw_body.decode('utf-8')
        except:
            return raw_body
    return body

def get_last_response_body(msmt):
    try:
        # The requests/response list sorts them from the newest to the oldest, 
        # hence the first item in the list is the last response we received.
        body = msmt['test_keys']['requests'][0]['response']['body']
        return decode_body(body)
    except (KeyError, TypeError, IndexError):
        return ''

TITLE_REGEXP = re.compile("<title.*?>(.*?)</title>", re.IGNORECASE | re.DOTALL)
# Doesn't take into account ordering
META_TITLE_REGEXP = re.compile("<meta.*?property=\"og:title\".*?content=\"(.*?)\"", re.IGNORECASE | re.DOTALL)

def get_http_title(msmt):
    body = get_last_response_body(msmt)
    # If the body is not a str object, it means it's binary (or an encoding we could not detect). 
    # No point in trying to extract the title.
    # Handling it like this is not very clean or nice.
    if not isinstance(body, str):
        return ''

    m = TITLE_REGEXP.search(body, re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1)
    return ''

    return extract_title(get_last_response_body(msmt))

def get_meta_http_title(msmt):
    body = get_last_response_body(msmt)
    if not isinstance(body, str):
        return ''

    m = META_TITLE_REGEXP.search(body, re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1)
    return ''

def get_http_body_hash(msmt):
    body = get_last_response_body(msmt)
    if body == '':
        return ''
    if isinstance(body, str):
        # We need the content of the body to be binary.
        body = body.encode('utf-8')
    return hashlib.md5(body[:2048]).hexdigest()

def base_metadata(msmt):
    base_keys = [
        'input',
        'measurement_start_time',
        'probe_asn',
        'probe_cc',
        'probe_network_name',
        'report_id',
        'resolver_asn',
        'resolver_ip',
        'resolver_network_name',
        'software_name',
        'software_version',
        'test_name',
        'test_runtime',
        'test_version'
    ]
    base_metadata = {}
    for k in base_keys:
        base_metadata[k] = msmt.get(k, '')
    annotations = msmt.pop('annotations')
    base_metadata['network_type'] = annotations.get('network_type', 'unknown')
    base_metadata['origin'] = annotations.get('origin', 'unknown')
    base_metadata['platform'] = annotations.get('platform', 'unknown')
    return base_metadata

In [4]:
def get_measurement_meta(msmt):
    m = base_metadata(msmt)
    m.update(get_resolver_info(msmt))
    m.update({
        'dns_resolved_ips': get_resolved_ips(msmt),
        'network_events': get_network_events(msmt),
        'control_failure': get_control_failure(msmt),
        'control_measurement': msmt['test_keys']['control'],
        'blocking': get_test_keys_blocking(msmt),
        'http_experiment_failure': get_http_experiment_failure(msmt),
        'dns_experiment_failure': str(msmt['test_keys']['dns_experiment_failure']),
        'http_title': get_http_title(msmt),
        'http_meta_title': get_meta_http_title(msmt),
        'http_body_md5': get_http_body_hash(msmt),
        'tcp_connect': get_tcp_connect(msmt),
    })
    return m

### Parsing raw files on disk, filtering and transforming them

Below are functions that will list the files on disk, given a search query, and return an iterator of the raw measurement dict.

These functions are then called by either `msmt_to_csv` or `get_msmt_df`, which write the processed data to a CSV file or load it in memory as a pandas DataFrame respectively.

It's generally recommended, when you are dealing with very large datasets, to write the minimised form of the data to a file on disk so that you don't have to re-parse everything if your notebook crashes.

In [5]:
from tqdm import tqdm
from pathlib import Path
import gzip
import ujson

data_dir = Path("/home/sitinurliza/projects/imap/ooni-data")

def iter_msmts(fp):
    with gzip.open(fp) as in_file:
        yield from [ujson.loads(line) for line in in_file]
            
def iter_jsonl_paths(query):
    for p in data_dir.glob('*/*/*/*'):
        ts, cc, tn = p.name.split('_')
        tn = tn.split('.')[0]
        ts = datetime.strptime(ts, '%Y%m%d%H')
        if query.get('probe_cc') and cc != query['probe_cc']:
            continue
        if query.get('test_name') and tn != query['test_name'].replace('_', ''):
            continue
        if query.get('since') and parse_date(query['since']) >= ts:
            continue
        if query.get('until') and parse_date(query['until']) <= ts:
            continue
        yield p
        
def iter_raw_measurements(query):
    path_list = list(iter_jsonl_paths(query))
    print(f"processing {len(path_list)}")
    for fp in tqdm(path_list):
        for msmt in iter_msmts(fp):
            if query.get('probe_asn') and msmt['probe_asn'] != query['probe_asn']:
                continue
            if query.get('domain'):
                domain = urlparse(msmt['input']).netloc
                if domain != query['domain']:
                    continue
            yield msmt

In [6]:
import csv

def msmt_to_csv(query, output_file="output.csv"):
    with open(output_file, 'w') as output_file:
        csv_writer = None
        for msmt in iter_raw_measurements(query):
            msmt_meta = get_measurement_meta(msmt)
            if csv_writer is None:
                fieldnames = msmt_meta.keys()
                csv_writer = csv.DictWriter(output_file, fieldnames=fieldnames)
                csv_writer.writeheader()
            csv_writer.writerow(msmt_meta)

In [7]:
def get_msmt_df(query):
    msmt_list = []
    for msmt in iter_raw_measurements(query):
        mdf = pd.DataFrame([get_measurement_meta(msmt)])
        msmt_list.append(mdf)
    return pd.concat(msmt_list, ignore_index=True)

Here we do the actual conversion to CSV. Edit the dates and country codes accordingly.

In [ ]:
msmt_to_csv({
    'since': '2022-01-01',
    'until': '2022-12-31',
    'probe_cc': 'MM',
    'test_name': 'web_connectivity'
}, output_file="ooni-data-wc-myanmar-2023-1.csv")

In [ ]:
msmt_to_csv({
    'since': '2023-01-01',
    'until': '2023-05-31',
    'probe_cc': 'MM',
    'test_name': 'web_connectivity'
}, output_file="ooni-data-wc-myanmar-2023-2.csv")

In [8]:
msmt_to_csv({
    'since': '2023-06-01',
    'until': '2023-07-01',
    'probe_cc': 'MM',
    'test_name': 'web_connectivity'
}, output_file="ooni-data-wc-myanmar-2023-3.csv")

processing 552


100%|████████████████████████████████████████████████████████████████████████████████| 552/552 [06:09<00:00,  1.50it/s]


In [8]:
!wc -l ooni-data-wc-myanmar.csv #counts characters in the files

936082 ooni-data-wc-myanmar.csv


We then load the CSV file in memory as a pandas dataframe for more analysis

In [9]:
df_mm_1 = pd.read_csv('ooni-data-wc-myanmar-2023-1.csv')

In [10]:
df_mm_2 = pd.read_csv('ooni-data-wc-myanmar-2023-2.csv')

In [11]:
df_mm_3 = pd.read_csv('ooni-data-wc-myanmar-2023-3.csv')

In [12]:
df_mm = pd.concat([df_mm_1,df_mm_2, df_mm_3])

In [19]:
len(df_mm)

1192747

In [14]:
list(df_mm)

['input',
 'measurement_start_time',
 'probe_asn',
 'probe_cc',
 'probe_network_name',
 'report_id',
 'resolver_asn',
 'resolver_ip',
 'resolver_network_name',
 'software_name',
 'software_version',
 'test_name',
 'test_runtime',
 'test_version',
 'network_type',
 'origin',
 'platform',
 'dns_resolved_ips',
 'network_events',
 'control_failure',
 'control_measurement',
 'blocking',
 'http_experiment_failure',
 'dns_experiment_failure',
 'http_title',
 'http_meta_title',
 'http_body_md5',
 'tcp_connect']

### Adding columns

When dealing with websites, we generally care to look at data from a domain centric perspective. This allows us to group together URLs that are of the same domain, but that have different paths.

Since the raw dataset doesn't include the `domain` we add this column here.

In [15]:
def parse_domain(url):
    try:
        return urlparse(url).netloc
    except:
        print(f'invalid url {url}')
        return ''
df_mm['domain'] = df_mm['input'].progress_apply(parse_domain)

100%|█████████████████████████████████████████████████████████████████████| 1192747/1192747 [00:16<00:00, 70669.83it/s]


In [16]:
df_mm.memory_usage(deep=True).sum()/1024**3

5.59077185112983

In [17]:
df_mm['explorer_url'] = "https://explorer.ooni.org/measurement/" + df_mm['report_id'] + "?input=" + df_mm['input']

### Listing out all probe_asn names

In [18]:
df_mm[['probe_asn', 'probe_network_name']].drop_duplicates(subset=['probe_asn']).to_csv('probe_names_MM_2023.csv')

### Hunting for blocking fingerprints

We can have a very high confidence that the blocking is intentional (and not caused by transient network failures), when it fits in the following classes:
- DNS level interference
- HTTP level intereference
- TLS MITM


The first two classes, though, are susceptive to false positives, because sometimes the IP returned in a DNS query can differ based on the geographical location (think CDNs) and sometimes the content of a webpage can also vary from request to request (think the homepage of a news site).

On the other hand, once we find a blocking fingerprint, we can with great confidence claim that access to that particular site is being restricted. For example we might notice that when a site is blocked on a particular network, the DNS query always returns a given IP address or we might know that the HTTP title for a blockpage is always "Access to this website is denied".

Our goal now to come up with some heuristics that will allow us to, in a way, hunt for these blockpage fingerprints in the big dataset that we have available.

### Same title, but different page

One heuristic which we can apply to spotting blockpages, is that we can say that a web page that looks exactly the same for many different sites. Based on this fairly simple intuition, we can look for blockpage fingerprints by just counting for the number of domains that share the same HTTP title tag.

In [16]:
title_domain_count = df_mm[
    df_mm['blocking'] == 'http-diff'
].groupby('http_title')['domain'].nunique().sort_values().reset_index()

As we can see in the breakdown below, all these blockpage fingerprints look fairly suspicious and are quite likely to be an indication of blocking. Some of them, however, might be signs of server-side blocking (ex. Geoblocking or DDOS prevention). This is why it's best, to obtain a high degree of accuracy, to investigate these manually and add them to a fingerprint database.

This is a shared effort amonst censorship research projects, for example you can find a repo of known blocking fingerprints maintained by the CitizenLab here: https://github.com/citizenlab/filtering-annotations 

In [17]:
title_domain_count[
    title_domain_count['domain'] > 8
]

,http_title,domain
70,"One moment, please...",12


In [18]:
http_title_suspicious = title_domain_count[title_domain_count['domain'] > 8]['http_title'].values.tolist()
http_title_suspicious

['One moment, please...']

In [19]:
df_mm[df_mm['http_title'].isin(http_title_suspicious)].drop_duplicates(subset=['http_title']).to_csv('http_title_suspicious_MM_2023.csv')

In [20]:
body_domain_count = df_mm[
    df_mm['blocking'] == 'http-diff'
].groupby('http_body_md5')['domain'].nunique().sort_values().reset_index()

In [21]:
pd.set_option('display.max_colwidth', None)

body_domain_count[
    body_domain_count['domain'] > 8
]

,http_body_md5,domain
680,fc19d76654b9647c43e8d61701645aec,45


In [22]:
http_body_suspicious = body_domain_count[body_domain_count['domain'] > 8]['http_body_md5'].values.tolist()
http_body_suspicious

['fc19d76654b9647c43e8d61701645aec']

In [23]:
df_mm[df_mm['http_body_md5'].isin(http_body_suspicious)].drop_duplicates(subset=['http_body_md5']).to_csv('http_body_suspicious_MM_2023.csv')

Once we have confirmed that a fingerprint is known to implement blocking, we can use it to which domains are being restricted.

### DNS level interference

We can use a similar heuristics for DNS level interference. The assumption is the same, when we see one IP being mapped to multiple hostnames, it's an indication of it potentially being an IP used to implement blocking.

In this case, we need to be careful of false positives that might be caused by the use of CDNs, as these will be hosting multiple sites. In the sections below we can see what techniques we can adopt to reduce these false positives further.

We are going to make use of a IP to ASN database for some of our heuristics. In particular we are going to download the one from db-ip, which has a fairly permissive license and is compatible with the maxmind database format.

In [17]:
!curl -O https://download.db-ip.com/free/dbip-asn-lite-2023-06.mmdb.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4031k  100 4031k    0     0   241k      0  0:00:16  0:00:16 --:--:--  280k     0  0:00:20  0:00:05  0:00:15  251k


In [21]:
!gunzip dbip-asn-lite-2023-06.mmdb.gz

In [24]:
import maxminddb

asn_db_path = 'dbip-asn-lite-2023-06.mmdb'
def lookup_asn(ip):
    with maxminddb.open_database(asn_db_path) as reader:
        try:
            return reader.get(ip)
        # Probably not an IP
        except ValueError:
            return None

In [25]:
dns_resp_sorted = df_mm[
    df_mm['blocking'] == 'dns'
].groupby('dns_resolved_ips')['domain'].nunique().sort_values().reset_index()

In [26]:
dns_resp_sorted[
    dns_resp_sorted['domain'] > 2
]

,dns_resolved_ips,domain
377,['146.112.61.106'],3
378,['99.83.154.118'],3
379,['213.156.224.213'],3
380,['70.32.1.32'],4
381,['34.117.168.233'],6
382,['59.153.90.11'],7
383,['74.125.68.121'],14
384,['0.0.0.0'],19
385,['103.105.174.6'],45
386,['172.31.31.2'],47


In [27]:
dns_resp_sorted[
    dns_resp_sorted['domain'] > 10
]

,dns_resolved_ips,domain
383,['74.125.68.121'],14
384,['0.0.0.0'],19
385,['103.105.174.6'],45
386,['172.31.31.2'],47
387,['167.172.4.60'],88
388,['127.0.0.1'],277
389,[],1734


In [28]:
dns_suspicious = dns_resp_sorted[dns_resp_sorted['domain'] > 10]['dns_resolved_ips'].values.tolist()
dns_suspicious

["['74.125.68.121']",
 "['0.0.0.0']",
 "['103.105.174.6']",
 "['172.31.31.2']",
 "['167.172.4.60']",
 "['127.0.0.1']",
 '[]']

In [29]:
df_mm[df_mm['dns_resolved_ips'].isin(dns_suspicious)].drop_duplicates(subset=['dns_resolved_ips','probe_asn']).to_csv('dns_suspicious_MM_2023.csv')

### DNS inconsistency false positive removal

To understand if what we are looking at is a real blocking IP or not, we can use the following heuristics:

1. Does the IP in question have a PTR record pointing to something that looks like a blockpage (ex. a hostname that is related to the ISP)
2. What information can we get about the IP by doing a whois lookup
3. Is the ASN of the IP the same as the network where the measurement was collected
4. Do we get a valid TLS certificate for one of the domains in question when doing a TLS handshake and specifying the SNI

Using these 4 conditions, we are generally able to understand if it's in fact a blocking IP or not

### Categorizing IPs into confirmed positives and false positives, as well as http titles that are confirmed positives...

In [20]:
confirmed_ips = [
'167.172.4.60',
'103.105.174.6',
]

false_positive_ips = [
'74.125.68.121'
]

confirmed_titles = [
]

confirmed_bodies = [
'fc19d76654b9647c43e8d61701645aec'
]

In [21]:
valid_ip_map = {}

In [8]:
import certifi
import ssl
import socket

def is_tls_valid(ip, hostname):
    if len(df_mm[
        (df_mm['dns_resolved_ips'].str.contains(ip, na=False))
        & (df_mm['domain'] == hostname)
        & (df_mm['input'].str.startswith('https'))
        & (df_mm['http_experiment_failure'] == 'None')
    ]) > 0:
        return True

    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.load_verify_locations(certifi.where())

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0) as sock:
        sock.settimeout(1)
        with context.wrap_socket(sock, server_hostname=hostname) as conn:
            try:
                conn.connect((ip, 443))
            # TODO: do we care to distinguish these values?
            except ssl.SSLCertVerificationError:
                return False
            except ssl.SSLError:
                return False
            except socket.timeout:
                return False
            except socket.error:
                return False
            except:
                return False
    return True

def is_tls_valid_with_cache(ip, hostname):
    key = f"{ip}{hostname}"
    if key in valid_ip_map:
        return valid_ip_map[key]
    valid_ip_map[key] = is_tls_valid(ip, hostname)
    return valid_ip_map[key]

### Putting it all together

We can then proceed to automating the detection on the full dataset. Our goal is that of recomputing the `blocking` feature for each individual measurement based on our improved heuristics.

In addition to the previously discussed DNS and HTTP based blocking, we are going to additionally classify blocking that happens at different layers of the network stack.

Specifically, we are going to be using the following identifiers for the various ways in which blocking might occur:

#### DNS
* dns.confirmed - one of the returned IPs matches an IP known to be used to implement blocking
* dns.no_ipv4 - no IPv4 address was returned
* dns.bogon - a bogon IP address was returned
* dns.nxdomain - we got an NXDOMAIN response from the probe, but we got a valid response from the control vantage point
* dns.inconsistent - our DNS consistency heuristics determined the returned IP to be inconsistent

#### HTTP

These are all blocking types related to plaintext HTTP requests:

* http.confirmed - the returned page is a known blockpages
* http.http_diff - the page doesn't match based on our page consistency heuristics
* http.connection_reset - we got a connection reset to a plaintext HTTP request
* http.connection_closed - the connection was closed before all data was transmitted
* http.connection_timeout - the connection timed out before we could retrieve all the data 
* http.generic_failure - this is an generic error from legacy OONI probes

#### TLS

These are all blocking types related to TLS:

* tls.connection_reset - a reset packet was seen after the client sent the ClientHello packet
* tls.connection_closed - the connection was closed after the ClientHello
* tls.connection_timeout - the connection timed out after the ClientHello
    * All of the above can also have the `_after_hello` suffix, indicating that the event happened after the client sent the ClienHello packet
* tls.mitm - The DNS is consistent, but the TLS certificate validation failed. This suggest a TLS man-in-the-middle
* tls.generic_failure - generic error from legacy OONI probes

#### TCP/IP

This is when blocking is implemented by targeting the IP address of the host:

* tcp.connection_reset - the TCP connect test failed due to a reset packet
* tcp.connection_timeout - the TCP connect test failed with a timeout

In [22]:
from ast import literal_eval
import ipaddress

def normalize_failure(failure_str):
    if "An existing connection was forcibly closed by the remote host" in failure_str:
        return "connection_reset"
    if "No address associated with hostname" in failure_str:
        return "dns_nxdomain_error"
    return failure_str

def is_dns_asns_consistent(dns_resolved_ips, control_measurement, row):
    try:
        control_addrs = control_measurement['dns']['addrs']
        if not control_addrs:
            return False
        control_asns = set(list(map(lambda e: e['autonomous_system_number'], 
                           filter(lambda e: e != None, map(lookup_asn, control_addrs)))))
        exp_asns = set(list(map(lambda e: e['autonomous_system_number'], 
                           filter(lambda e: e != None, map(lookup_asn, dns_resolved_ips)))))
        if exp_asns.intersection(control_asns):
            return True
    except KeyError:
        # Missing control measurement
        return False
    return False

bogon_ipv4_ranges = [
    ipaddress.ip_network("0.0.0.0/8"), # "This" network
    ipaddress.ip_network("10.0.0.0/8"), # Private-use networks
    ipaddress.ip_network("100.64.0.0/10"), # Carrier-grade NAT
    ipaddress.ip_network("127.0.0.0/8"), # Loopback
    ipaddress.ip_network("127.0.53.53"), # Name collision occurrence
    ipaddress.ip_network("169.254.0.0/16"), # Link local
    ipaddress.ip_network("172.16.0.0/12"), # Private-use networks
    ipaddress.ip_network("192.0.0.0/24"), # IETF protocol assignments
    ipaddress.ip_network("192.0.2.0/24"), # TEST-NET-1
    ipaddress.ip_network("192.168.0.0/16"), # Private-use networks
    ipaddress.ip_network("198.18.0.0/15"), # Network interconnect device benchmark testing
    ipaddress.ip_network("198.51.100.0/24"), # TEST-NET-2
    ipaddress.ip_network("203.0.113.0/24"), # TEST-NET-3
    ipaddress.ip_network("224.0.0.0/4"), # Multicast
    ipaddress.ip_network("240.0.0.0/4"), # Reserved for future use
    ipaddress.ip_network("255.255.255.255/32"), # Limited broadcast
]
def is_dns_bogon(dns_resolved_ips):
    for ip in dns_resolved_ips:
        ipv4addr = ipaddress.IPv4Address(ip)
        if any([ipv4addr in ip_range for ip_range in bogon_ipv4_ranges]):
            return True
    return False

def is_dns_tls_consistent(dns_resolved_ips, row):
    # If it's a HTTPs site and we didn't get a TLS error, we can assume the IPs are valid
    if row['input'].startswith('https://') and row['http_experiment_failure'] == 'None':
        return False
    
    for ip in dns_resolved_ips:
        domain = urlparse(row['input']).netloc
        if is_tls_valid_with_cache(ip, domain):
            # We consider the first hit to be enough to consider it consistent
            return True
    return False

def is_dns_false_positive(dns_resolved_ips):
    for ip in dns_resolved_ips:
        if ip in false_positive_ips:
            return True
    return False

def recompute_blocking(row):
    if not isinstance(row['input'], str):
        return 'invalid'
    
    try:
        dns_resolved_ips = literal_eval(row['dns_resolved_ips'])
    except:
        dns_resolved_ips = []

    blocking = row['blocking']
    for ip in dns_resolved_ips:
        if ip in confirmed_ips:
            return 'dns.confirmed'
        
    # This is a special case for when we got no ipv4 addresses and the network doesn't support ipv6
    if len(dns_resolved_ips) == 0 and row['http_experiment_failure'] == 'network_unreachable':
        return 'dns.no_ipv4'
    
    if is_dns_bogon(dns_resolved_ips):
        return 'dns.bogon'

    try:
        control_measurement = literal_eval(row['control_measurement'])
    except:
        return 'invalid'
    if not control_measurement:
        return 'invalid'
    
    if control_measurement['http_request']['failure'] != None:
        return 'invalid'

    if (normalize_failure(row['dns_experiment_failure']) == 'dns_nxdomain_error' and 
            control_measurement.get('http_request', {}).get('failure', '') != 'dns_lookup_error'):
        return 'dns.nxdomain'

    if (
        not (row['input'].startswith('https://') and row['http_experiment_failure'] == 'None') 
        and not is_dns_false_positive(dns_resolved_ips) 
        and not is_dns_asns_consistent(dns_resolved_ips, control_measurement, row)
        #and not is_dns_tls_consistent(dns_resolved_ips, row)
    ):
        return 'dns.inconsistent'

    # If we got down to here, it means that DNS is consistent    
    if row['http_title'] in confirmed_titles:
        return 'http.title.confirmed'
    
    if row['http_body_md5'] in confirmed_bodies:
        return 'http.body.confirmed'
    
    if blocking == 'http-diff' and row['input'].startswith('http://'):
        return 'http.http_diff'
    
    if row['http_experiment_failure'] != 'None':
        tcp_connect_list = literal_eval(row['tcp_connect'])
        for conn in tcp_connect_list:
            if conn['status']['failure'] == 'connection_reset':
                return 'tcp.connection_reset'
            elif conn['status']['failure'] == 'generic_timeout_error':
                return 'tcp.connection_timeout'
    
    # We compute TLS level anomalies this using the network_events
    tls_handshake_started = False
    try:
        network_events = literal_eval(row['network_events'])
    except:
        network_events = []
    if network_events:
        for idx, network_event in enumerate(network_events):
            if network_event['operation'] == 'write':
                write_operations += 1
            if network_event['operation'] == 'read':
                read_operations += 1

            if tls_handshake_started and network_event['failure']:
                # We are guaranteed to not be out of bounds due to the tls_handshake_started flag
                prev_operation = network_events[idx-1]
                
                suffix = ''
                if normalize_failure(network_event['failure']) == 'connection_reset':
                    return f'tls.connection_reset{suffix}'
                elif normalize_failure(network_event['failure']) == 'eof_error':
                    return f'tls.connection_closed{suffix}'
                elif normalize_failure(network_event['failure']) == 'generic_timeout_error':
                    return f'tls.connection_timeout{suffix}'
                if write_operations > 1:
                    suffix = f'_after_hello'

            if network_event['operation'] == 'tls_handshake_start':
                tls_handshake_started = True
                write_operations = 0
                read_operations = 0
            if network_event['operation'] == 'tls_handshake_done':
                tls_handshake_started = False

    # If we got down to here, it means the DNS consistency checks have passed
    # For the http related failures, if we are spotting them here, it means the test most likely doesn't support the 
    # new network_events keys, and therefore the results are a bit less accurate.
    # This should ideally be indicated via a lower confidence value.
    if normalize_failure(row['http_experiment_failure']) == 'connection_reset':
        if row['input'].startswith('https://'):
            return 'tls.connection_reset'
        else:
            return 'http.connection_reset'
    elif normalize_failure(row['http_experiment_failure']) == 'eof_error':
        if row['input'].startswith('https://'):
            return 'tls.connection_closed'
        else:
            return 'http.connection_closed'
    elif normalize_failure(row['http_experiment_failure']) == 'generic_timeout_error':
        if row['input'].startswith('https://'):
            return 'tls.connection_timeout'
        else:
            return 'http.connection_timeout'
    # It's not just using DNS to point us to an IP that serves a blockpage and it's a TLS MITM
    elif row['input'].startswith('https://') and row['http_experiment_failure'].startswith('ssl_'):
        return 'tls.mitm'
    
    # We map unknown_failures to invalid measurements
    elif row['http_experiment_failure'].startswith('unknown_failure'):
        return 'invalid'
    
    # All unmapped errors go into a generic failure pool
    elif row['http_experiment_failure'] != 'None':
        if row['input'].startswith('https://'):
            return 'tls.generic_failure'
        else:
            return 'http.generic_failure'
    
    return 'ok'

In [25]:
df_mm['blocking_recalc'] = df_mm.progress_apply(recompute_blocking, axis=1)

100%|███████████████████████████████████████████████████████████████████████| 1192747/1192747 [23:36<00:00, 841.86it/s]


In [26]:
df_mm['blocking_recalc'].unique()

array(['dns.inconsistent', 'ok', 'tcp.connection_timeout', 'invalid',
       'tls.connection_timeout', 'http.http_diff', 'tls.connection_reset',
       'dns.nxdomain', 'tls.mitm', 'http.connection_closed',
       'http.connection_timeout', 'dns.bogon', 'tls.connection_closed',
       'tls.generic_failure', 'dns.confirmed', 'dns.no_ipv4',
       'http.connection_reset', 'http.generic_failure',
       'tcp.connection_reset', 'http.body.confirmed'], dtype=object)

In [27]:
df_mm[
    df_mm['blocking_recalc'] == 'dns.inconsistent'
]['dns_resolved_ips'].unique()

array(["['157.240.235.174']", "['104.244.42.65', '104.244.42.1']",
       "['157.240.235.35']", ...,
       "['35.161.191.171', '54.189.1.74', '34.214.28.25']",
       "['45.41.96.108', '45.41.96.109', '45.41.96.112', '45.41.96.111', '45.41.96.113', '45.41.96.107', '45.41.96.110', '45.41.96.114']",
       "['54.189.1.74', '35.161.191.171', '34.214.28.25']"], dtype=object)

Let's see on how many networks we were able to confirm the blocking of sites

In [29]:
df_mm[
    df_mm['blocking_recalc'] == 'dns.confirmed'
]['probe_asn'].unique()

array(['AS133385', 'AS136780', 'AS134840', 'AS133440'], dtype=object)

In [31]:
df_mm[
    df_mm['blocking_recalc'] == 'http.title.confirmed'
]['probe_asn'].unique()

array([], dtype=object)

In [32]:
df_mm[
    df_mm['blocking_recalc'] == 'http.body.confirmed'
]['probe_asn'].unique()

array(['AS135300'], dtype=object)

In [50]:
msmt_counts = df_mm[
    (df_mm['blocking_recalc'] == 'dns.confirmed') | 
    (df_mm['blocking_recalc'] == 'http.title.confirmed') | 
    (df_mm['blocking_recalc'] == 'http.body.confirmed')
][['input', 'blocking_recalc', 'report_id']].groupby(['input', 'blocking_recalc']).count().reset_index()

In [62]:
confirmed_blocking = df_mm[
    (df_mm['blocking_recalc'] == 'dns.confirmed') | 
    (df_mm['blocking_recalc'] == 'http.title.confirmed') | 
    (df_mm['blocking_recalc'] == 'http.body.confirmed')]

msmt_counts = confirmed_blocking.pivot_table(index=['input'], values=['report_id', 'blocking_recalc', 'probe_asn'],
                                                aggfunc={'report_id': len, 
                                                         'blocking_recalc': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'report_id': 'blocking_count',
                                                                    'blocking_recalc' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [63]:
print(msmt_counts.head())

                                input      blocking_method probe_asn_blocking  \
0              http://anonymouse.org/        dns.confirmed           AS133385   
1              http://aungsanu.com/my        dns.confirmed           AS133385   
2     http://auntysweet.blogspot.com/  http.body.confirmed           AS135300   
3  http://blog-aunghtut.blogspot.com/  http.body.confirmed           AS135300   
4              http://blog.mghla.net/  http.body.confirmed           AS135300   

   blocking_count  
0               9  
1               5  
2               1  
3               1  
4               1  


In [64]:
msmt_counts.to_csv('2023-myanmar-confirmed-by-heuristics')

We will now reshape the data using `pivot_table` and export the data of `blocking_recalc` into Google Data Studio. 

In [65]:
data_export = df_mm[['input',
 'measurement_start_time',
 'probe_asn',
 'probe_cc',
 'probe_network_name',
 'report_id',
 'resolver_asn',
 'resolver_ip',
 'resolver_network_name',
 'software_name',
 'software_version',
 'test_name',
 'test_runtime',
 'test_version',
 'network_type',
 'origin',
 'platform',
 'dns_resolved_ips',
 'network_events',
 'control_failure',
 'control_measurement',
 'blocking',
 'http_experiment_failure',
 'dns_experiment_failure',
 'http_title',
 'http_meta_title',
 'http_body_md5',
 'tcp_connect',
 'domain',
 'explorer_url',
 'blocking_recalc']]

In [67]:
data_export['count'] = 1

At this point we would iterate the process of filtering out any additional false positives and false negatives, until we feel quite confident that we have eliminated most of the outliers (or come up with an explaination as to why we are seeing them).

Once this process is done, it might be desirable to create a CSV export of this cleaned data in preparation for publication ready charts (ex. through tools like Tableau).

Since charting tools generally work best with data where the values you need to plot are in the cells and the columns indicate the category of the value, we will reshape the data using the `pivot_table` function. This basically takes the values of `blocking_recalc` and puts them as columns, the value of the cells, in this case, is always going to be one. It's generally quite easy to do further aggregation and grouping inside of the charting tool itself.

In [68]:
data_export.pivot_table(
    index=['input',
 'measurement_start_time',
 'probe_asn',
 'probe_cc',
 'probe_network_name',
 'report_id',
 'resolver_asn',
 'resolver_ip',
 'resolver_network_name',
 'software_name',
 'software_version',
 'test_name',
 'test_runtime',
 'test_version',
 'network_type',
 'origin',
 'platform',
 'dns_resolved_ips',
 'network_events',
 'control_failure',
 'control_measurement',
 'blocking',
 'http_experiment_failure',
 'dns_experiment_failure',
 'http_title',
 'http_meta_title',
 'http_body_md5',
 'tcp_connect',
 'domain',
 'explorer_url'], 
    columns=['blocking_recalc'],
    values='count'
).reset_index().to_csv('2023-myanmar-websites.csv')

In [46]:
!wc -l 2023-myanmar-websites.csv

46874 2023-myanmar-websites.csv


In [47]:
!du -sch 2023-myanmar-websites.csv

94M	2023-myanmar-websites.csv
94M	total
